* 条件：经过中国33个城市，一共4辆车，每辆车最大行驶10000公里
* 目标：使得每辆车的行驶里程数更接近

In [2]:
import pandas as pd
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [3]:
pd.read_excel('./cities.xlsx').T

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
name,北京,天津,哈尔滨,长春,沈阳,石家庄,呼和浩特,太原,上海,杭州,...,成都,重庆,南宁,贵阳,昆明,武汉,长沙,南昌,香港,澳门
location,"116.403039,39.914909","117.216419,39.143831","126.637622,45.766187","125.330606,43.918866","123.401674,41.800522","114.470479,38.074112","111.671565,40.837895","112.594097,37.86707","121.480324,31.236546","120.18972,30.248403",...,"104.064264,30.66394","106.554988,29.555927","108.321874,22.832993","106.709874,26.563191","102.728219,25.022114","114.308774,30.548863","112.94547,28.235017","115.924376,28.669681","114.17495,22.327115","113.54814,22.181782"


In [4]:
pd.read_excel('./distance.xlsx', index_col=0).head(3)

,北京,天津,哈尔滨,长春,沈阳,石家庄,呼和浩特,太原,上海,杭州,...,成都,重庆,南宁,贵阳,昆明,武汉,长沙,南昌,香港,澳门
北京,0,122476,1229869,987941,682963,294125,491658,487353,1210971,1277042,...,1835076,1753571,2346302,2086066,2587861,1161356,1477547,1431143,2203897,2273144
天津,122476,0,1201742,959814,654836,314982,614846,508210,1093989,1217026,...,1855933,1774428,2343851,2083615,2585410,1158905,1475096,1358857,2201446,2270693
哈尔滨,1229869,1201742,0,244488,551085,1493810,1684809,1687038,2269863,2392900,...,3034761,2953256,3533141,3272905,3774700,2348195,2664386,2534550,3390736,3459983


In [30]:
class tsp(object):
    def __init__(self, city_names=None, init_opt_strategy="PATH_CHEAPEST_ARC", local_search_strategy="GUIDED_LOCAL_SEARCH"):
        # 获取城市的名称，经纬度
        self.df = pd.read_excel('./cities.xlsx')
        self.all_city = self.df['name'].values
        self.init_opt_strategy = init_opt_strategy
        self.local_search_strategy = local_search_strategy
        
        if city_names is not None:
            self.city_names = city_names
            # 如果不筛选df，下标会不对应 北京0 天津1 上海2
            # 筛选出来有city_names的行和列
            self.df = self.df[self.df['name'].isin(city_names)]
        else:
            self.city_names = self.all_city
        
    # 设置数据
    def create_data_model(self):
        temp = pd.read_excel('./distance.xlsx', index_col=0)
        # 按照city_names进行筛选 两点之间的距离
        temp = temp[(temp.index.isin(self.city_names))][self.city_names]
        
        data = {}
        # 设置3个维度： 邻接矩阵（城市之间距离），车的数量，起始点
        data['distance_matrix'] = temp.values / 1000 # 采用公里计算
        data['num_vehicles'] = 4
        data['depot'] = 0 #起始点的ID，从北京出发
        return data
        
    # 输出路径规划
    def print_solution(self, data, manager, routing, solution):
        total_distance = 0
        # 遍历每辆车
        for vehicle_id in range(data['num_vehicles']):
            # 从这辆车开始
            index = routing.Start(vehicle_id)
            plan_output = '车{} 的行驶路径:\n'.format(vehicle_id)
            route_distance = 0
            while not routing.IsEnd(index):
                node_index = manager.IndexToNode(index)
                plan_output += ' {0} -> '.format(node_index)
                previous_index = index
                index = solution.Value(routing.NextVar(index))
                route_distance += routing.GetArcCostForVehicle(
                    previous_index, index, vehicle_id)
            plan_output += ' {0}\n'.format(manager.IndexToNode(index))
            plan_output += '路径距离: {}公里\n'.format(route_distance)
            print(plan_output)
            total_distance += route_distance
        print('所有路径的总距离: {}公里'.format(total_distance))

    def main(self):
        # 初始化数据
        data = self.create_data_model()

        # 创建路线管理，tsp_size (邻接矩阵表的大小), num_vehicles（车的数量）, depot（原点）
        manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                               data['num_vehicles'], data['depot'])

        # 创建路径规划模型 Routing Model.
        routing = pywrapcp.RoutingModel(manager)

        # 计算两点之间的距离
        def distance_callback(from_index, to_index):
            # 从Index => NodeIndex的转换
            # Convert from routing variable Index to distance matrix NodeIndex.
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            # 从distance_matrix中的起点、终点 找到距离
            return data['distance_matrix'][from_node][to_node]

        # 创建距离回调函数，用于计算两个节点之间的距离
        transit_callback_index = routing.RegisterTransitCallback(distance_callback)
        # 对所有的车辆，采用相同的distance callback的计算
        routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
        
        """添加距离约束"""
        dimension_name = 'Distance'
        routing.AddDimension(
            transit_callback_index,
            0,  # no slack
            10000,  # 车辆最大行驶距离
            True,  # start cumul to zero
            dimension_name)
        distance_dimension = routing.GetDimensionOrDie(dimension_name)
        # 尽量减少车辆之间的最大距离
        distance_dimension.SetGlobalSpanCostCoefficient(50)

        # 设置启发式方法（距离成本最短 PATH_CHEAPEST_ARC）
        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        # 初始化的解
        search_parameters.first_solution_strategy = (
            getattr(routing_enums_pb2.FirstSolutionStrategy, self.init_opt_strategy))
        # 在初始解的基础上，进一步优化（通过邻居调整）
        search_parameters.local_search_metaheuristic = (
            getattr(routing_enums_pb2.LocalSearchMetaheuristic, self.local_search_strategy))
        search_parameters.time_limit.seconds = 5

        #print(routing)
        # 求解路径规划
        solution = routing.SolveWithParameters(search_parameters)
        # 输出结果
        if solution:
            self.print_solution(data, manager, routing, solution)

In [25]:
# 采用不同的初始优化策略（随便挑了几个策略）
for init_opt_strategy in ["AUTOMATIC", "PATH_CHEAPEST_ARC", "PATH_MOST_CONSTRAINED_ARC", "SAVINGS", "GLOBAL_CHEAPEST_ARC"]:  
    print(f"采用的初始优化策略：{init_opt_strategy}")
    model = tsp(init_opt_strategy=init_opt_strategy)
    model.main()
    print("\n")

采用的初始优化策略：AUTOMATIC
车0 的行驶路径:
 0 ->  7 ->  18 ->  19 ->  17 ->  22 ->  6 ->  0
路径距离: 6341公里

车1 的行驶路径:
 0 ->  21 ->  0
路径距离: 7096公里

车2 的行驶路径:
 0 ->  5 ->  20 ->  23 ->  24 ->  26 ->  27 ->  25 ->  12 ->  32 ->  11 ->  31 ->  30 ->  13 ->  16 ->  0
路径距离: 6749公里

车3 的行驶路径:
 0 ->  14 ->  28 ->  29 ->  10 ->  9 ->  8 ->  15 ->  1 ->  4 ->  2 ->  3 ->  0
路径距离: 6845公里

所有路径的总距离: 27031公里


采用的初始优化策略：PATH_CHEAPEST_ARC
车0 的行驶路径:
 0 ->  7 ->  18 ->  19 ->  17 ->  22 ->  6 ->  0
路径距离: 6341公里

车1 的行驶路径:
 0 ->  21 ->  0
路径距离: 7096公里

车2 的行驶路径:
 0 ->  5 ->  20 ->  23 ->  24 ->  26 ->  27 ->  25 ->  12 ->  32 ->  11 ->  31 ->  30 ->  13 ->  16 ->  0
路径距离: 6749公里

车3 的行驶路径:
 0 ->  14 ->  28 ->  29 ->  10 ->  9 ->  8 ->  15 ->  1 ->  4 ->  2 ->  3 ->  0
路径距离: 6845公里

所有路径的总距离: 27031公里


采用的初始优化策略：PATH_MOST_CONSTRAINED_ARC
车0 的行驶路径:
 0 ->  7 ->  19 ->  18 ->  17 ->  22 ->  6 ->  0
路径距离: 6341公里

车1 的行驶路径:
 0 ->  21 ->  0
路径距离: 7096公里

车2 的行驶路径:
 0 ->  5 ->  20 ->  23 ->  24 ->  26 ->  27 ->  25 ->  12 -

最好的结果还是27031公里。

In [29]:
# 采用不同的局部优化策略
for local_search_strategy in ["AUTOMATIC", "GREEDY_DESCENT", "GUIDED_LOCAL_SEARCH", "SIMULATED_ANNEALING", "TABU_SEARCH"]:  
    print(f"采用的局部优化策略：{local_search_strategy}")
    model = tsp(local_search_strategy=local_search_strategy)
    model.main()
    print("\n")

采用的局部优化策略：AUTOMATIC
车0 的行驶路径:
 0 ->  7 ->  18 ->  19 ->  17 ->  22 ->  6 ->  0
路径距离: 6341公里

车1 的行驶路径:
 0 ->  21 ->  0
路径距离: 7096公里

车2 的行驶路径:
 0 ->  5 ->  20 ->  23 ->  24 ->  26 ->  27 ->  25 ->  12 ->  32 ->  11 ->  31 ->  30 ->  13 ->  16 ->  0
路径距离: 6749公里

车3 的行驶路径:
 0 ->  14 ->  28 ->  29 ->  10 ->  9 ->  8 ->  15 ->  1 ->  4 ->  2 ->  3 ->  0
路径距离: 6845公里

所有路径的总距离: 27031公里


采用的局部优化策略：GREEDY_DESCENT
车0 的行驶路径:
 0 ->  7 ->  18 ->  19 ->  17 ->  22 ->  6 ->  0
路径距离: 6341公里

车1 的行驶路径:
 0 ->  21 ->  0
路径距离: 7096公里

车2 的行驶路径:
 0 ->  5 ->  20 ->  23 ->  24 ->  26 ->  27 ->  25 ->  12 ->  32 ->  11 ->  31 ->  30 ->  13 ->  16 ->  0
路径距离: 6749公里

车3 的行驶路径:
 0 ->  14 ->  28 ->  29 ->  10 ->  9 ->  8 ->  15 ->  1 ->  4 ->  2 ->  3 ->  0
路径距离: 6845公里

所有路径的总距离: 27031公里


采用的局部优化策略：GUIDED_LOCAL_SEARCH
车0 的行驶路径:
 0 ->  7 ->  18 ->  19 ->  17 ->  22 ->  6 ->  0
路径距离: 6341公里

车1 的行驶路径:
 0 ->  21 ->  0
路径距离: 7096公里

车2 的行驶路径:
 0 ->  5 ->  20 ->  23 ->  24 ->  26 ->  27 ->  25 ->  12 ->  32 -> 

猜测27031已经是（或很接近）全局最优解了，因此采用不同的策略得到相同的解且无法进一步优化。